<a href="https://colab.research.google.com/github/sccasutt/city/blob/master/Laufender_Prototyp_Swiss_ASR_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
""" Start from scratch"""
!pip install datasets>=1.13.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer
!git config --global credential.helper store
!apt install git-lfs
!pip install datasets[audio]
!pip install torchaudio
!pip install wandb

     |████████████████████████████████| 2.9 MB 8.6 MB/s 
     |████████████████████████████████| 3.3 MB 41.5 MB/s 
     |████████████████████████████████| 895 kB 38.9 MB/s 
     |████████████████████████████████| 596 kB 48.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 50 kB 5.2 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149863 sha256=9912b9fb9e8c823aa2fcef5df4b28e3e0deff8805f4a55de6fd4f1b85bb82a6a
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,12

In [2]:
import numpy as np
import librosa
import pandas as pd
import os
from datasets import ClassLabel
import random
from IPython.display import display, HTML
from datasets import load_dataset
from datasets import Dataset
import datasets
from transformers import Wav2Vec2Model, Wav2Vec2Config
import torchaudio
import wandb
import re
import json
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer
import IPython.display as ipd
import random
from datasets import load_metric

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
wandb.init(project="my-test-project", entity="scasutt")

wandb: Currently logged in as: scasutt (use `wandb login --relogin` to force relogin)


In [18]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [5]:
feature_dict = {"audio_path":datasets.Audio(sampling_rate=16_000),"text":datasets.Value("string")}
features = datasets.Features(feature_dict)

dataset = load_dataset("csv", 
                       data_files={"train":"train_data.csv",
                                   "test":"test_data.csv"},  
                       features = features
                      )
dataset = dataset.cast_column("audio_path", datasets.Audio(sampling_rate=16_000))

Using custom data configuration default-31ce8a93a3e85dd9


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-31ce8a93a3e85dd9/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset["train"]

Dataset({
    features: ['audio_path', 'text'],
    num_rows: 2175
})

In [6]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"].remove_columns(["audio_path"]))

,text
0,"So gemein wie de Felix esch, hätter ehm secher absechtlech gschadet."
1,"Wenn d'Tier os Nachzöcht chömme, halt ich Aquarie för relativ onproblematisch."
2,"Zäme metem Wegner foht de Rico ah e Poppe z'baschtle, wö sött usgseh wie d'Regine."
3,Hingege de Bärge no gehts hüfiger Schnee.
4,Zerscht esch met Champagner ond anderene Wi os de Champfagne Kampagne Handel betrebe worde.
5,De Volksvorschlag esch met 164'927 zo 129'336 Bachab gscheckt worde.
6,Als Bispel nennt er d'Gsechtserkennig als Chonde-Identifikation.
7,D'Klus be Aesch esch es beliebts Usflogsziel.
8,Nach nün Protescht het sech d'Zentralregierig en Peking zor Lag en Hongkong gösseret.
9,"Haftar werd hüfig metem ägyptische Sisi vergleche, wo fom Armeechef zom Präsidänt worde esch."


In [7]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\(\)\«\»\’\/\'\%\x08\&\_\–\d{10}\ê\é\à"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

dataset = dataset.map(remove_special_characters)

def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = dataset.map(extract_all_chars, 
                     batched=True, 
                     batch_size=-1, 
                     keep_in_memory=True, 
                     remove_columns=dataset.column_names["train"])

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))
vocab_dict

  0%|          | 0/2175 [00:00<?, ?ex/s]

  0%|          | 0/573 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

32


{'[PAD]': 31,
 '[UNK]': 30,
 'a': 28,
 'b': 8,
 'c': 1,
 'd': 10,
 'e': 23,
 'f': 11,
 'g': 2,
 'h': 16,
 'i': 3,
 'j': 6,
 'k': 24,
 'l': 26,
 'm': 4,
 'n': 27,
 'o': 7,
 'p': 19,
 'q': 5,
 'r': 21,
 's': 20,
 't': 25,
 'u': 18,
 'v': 12,
 'w': 0,
 'x': 15,
 'y': 13,
 'z': 22,
 '|': 9,
 'ä': 14,
 'ö': 17,
 'ü': 29}

In [8]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [9]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [10]:
rand_int = random.randint(0, len(dataset["train"]))

print(dataset["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(dataset["train"][rand_int]["audio_path"]["array"]), autoplay=True, rate=16000)

de sionpräsident christian constantin esch trotz verhältnismässig diplomatische wort ned ume begriff sportlechs versage omecho


In [11]:
rand_int = random.randint(0, len(dataset["train"]))

print("Target text:", dataset["train"][rand_int]["text"])
print("Input array shape:", np.asarray(dataset["train"][rand_int]["audio_path"]["array"]).shape)
print("Sampling rate:", dataset["train"][rand_int]["audio_path"]["sampling_rate"])

Target text: veli fachlüt händ am internationale tag fode ältere generation dshowrüm beseschtigt
Input array shape: (71247,)
Sampling rate: 16000


In [12]:
def prepare_dataset(batch):
    audio = batch['audio_path']

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset, remove_columns = dataset.column_names["train"], num_proc=4)

In [13]:
#CTC implementation

import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [14]:
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

In [21]:
#initial model
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    
)

model.freeze_feature_extractor()

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'project_hid.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_hid.bias', 'project_q.bias', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [22]:
output_dir = "Prototype_training"


training_args = TrainingArguments(
  output_dir=output_dir,
  push_to_hub=True,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=8,
  fp16=False,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=100,
  logging_steps=60,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  report_to="wandb"
)



trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)

Cloning https://huggingface.co/scasutt/Prototype_training into local empty directory.


In [23]:
 torch.cuda.empty_cache()

In [24]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 2175
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2176
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and

Step,Training Loss,Validation Loss,Wer
100,7.611400,13.075162,1.061166
200,5.087700,3.753353,1.000000
300,3.075800,3.396463,1.000000
400,3.020600,3.474363,1.000000
500,2.996800,3.331644,1.000000
600,2.981100,3.164053,1.000000
700,2.954200,2.926457,1.000000
800,2.595900,2.068972,1.000000
900,1.606600,1.250532,0.901529
1000,1.339800,0.787798,0.765586


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 573
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 573
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 573
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 573
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_le

TrainOutput(global_step=2176, training_loss=1.9020329275551964, metrics={'train_runtime': 9093.1866, 'train_samples_per_second': 1.914, 'train_steps_per_second': 0.239, 'total_flos': 1.9532297268914102e+18, 'train_loss': 1.9020329275551964, 'epoch': 8.0})

In [25]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [26]:
results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

  0%|          | 0/573 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [30]:
show_random_elements(results)

,pred_str,text
0,tüfi fode fönschterbanka fo gebeude zue beude wariere,dtüfi fode fenschterbank cha fo gebäude zo gebäude variiere
1,wen situation onerträglech werd plant en handfol änner e usbroch,wen dsituation onerträglech werd plant en handvoll männer de usbroch
2,witerhin kompuliert het musig zu wer het öuser dine saurier klaut,wieterhen komponiert er dmusig zu wer het euse dinosaurier klaut
3,grondt dodefü sin tekste fode lieder gsi wo nachem petrozat totale blötzeengsisend,grond dodeför send dtexte fode lieder gsi wo nachem petrozza totale blödsenn gsi send
4,dzitwis esches nas am hüfigstem nordoscte ond am alpte ordong,zitwis esches nass am hüfigschte em nordoschte ond am alpenordhang
5,sie esch eck fom schef gsecklet,sie esch zom heck fom scheff gsecklet
6,dzuegleit het desspigmse rne uchrächenig allerdengs ned,zuegleiht de dsp gmäss dere hochrächnig allerdengs ned
7,dfinanzierig fom agebog erfo sponsore ond bannerwärbig,dfinanzierig vom agebot erfolgt dor sponsore ond bannerwärbig
8,ich tzee momänt gadiesalte huom,ich zieh em moment gad es alte hus om
9,bäld et gört si zode meischt sitierte forschend of erem gebit,weltwit ghört sie zode meischt zitierte forschende of erem gebiet


In [40]:
rand_int = random.randint(0, len(results["pred_str"]))

print(results[rand_int]["pred_str"])
ipd.Audio(data=np.asarray(dataset["test"][rand_int]["input_values"]), autoplay=True, rate=16000)

dänksc das öpper em ektraschadettt
